In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
os.getcwd()

'/Users/as685989/Github/cyclone-research'

In [3]:
#Import case metadata for each basin
wnp_metadata = pd.read_csv('subset_cyclones/wnp_era5_metadata.txt', sep = '\t')
atl_metadata = pd.read_csv('subset_cyclones/atl_era5_metadata.txt', sep = '\t')

In [6]:
#Preprocessing function to mimic NCL's join feature
def preproc(ds):
    da_new_time = xr.DataArray(np.array([0]), coords=[np.array([0])], dims='new_time')
    ds = ds.expand_dims('new_time').assign_coords(new_time=da_new_time).drop('time').squeeze('time').rename({'new_time': 'time'})    
    return ds

In [9]:
#Create a multifile xarray Dataset
dir_wnp = './ERA5_data/wnp/wnp_relative/' #Specify the directory containing file(s) of interest\
dir_atl = './ERA5_data/atl/atl_relative/'

DS_wnp = xr.open_mfdataset(dir_wnp+'wnp_*_rel.nc', preprocess = preproc,
                           combine = 'nested', concat_dim = 'case')
DS_wnp = DS_wnp.assign_coords(case=np.array(wnp_metadata['wnp_id']))

In [10]:
DS_atl = xr.open_mfdataset(dir_atl+'atl_*_rel.nc', preprocess = preproc,
                           combine = 'nested', concat_dim = 'case')
DS_atl = DS_atl.assign_coords(case=np.array(atl_metadata['atl_id']))

In [13]:
#Extract the geopotential variable
z_wnp = DS_wnp.z
z_atl = DS_atl.z

In [14]:
#Write a function to perform more preprocessing
def to_gph(da):
    gph = da/9.80665 #Convert geopotential to geopotential height
    gph.attrs['units'] = 'gpm'
    gph.attrs['long_name'] = '500 hPa Geopotential Height'
    return gph

In [15]:
gph_wnp = to_gph(z_wnp)
gph_atl = to_gph(z_atl)

In [18]:
#Weight the data based on latitude
def apply_weights(da):
    weight_arr = np.sqrt(np.cos(np.radians(da.latitude))) #sqrt of the cosine of the latitude in radians
    da_weighted = da * weight_arr #Apply the weights
    da_weighted.attrs = da.attrs #Copy attributes onto the weighted array
    return da_weighted

In [20]:
#Perform sqrt cos latitude weighting
gph_wnp_weighted = apply_weights(gph_wnp)
gph_atl_weighted = apply_weights(gph_atl)

In [25]:
#Function to extract a 4D numpy array of just the data values and
#reshape it so that it contains one flattened array for each case.
#Each case, once flattended, is 121*481, or 58,201 values

def to_input(da):
    input_values = da.values #Extract a 4D (case, time, latitude, longitude) numpy array of just the data values
    input_arr = input_values.reshape((input_values.shape[0], 58201)) #Reshape so that the input contains one flattened array for each case
    return input_arr

In [26]:
wnp_input = to_input(gph_wnp_weighted)
atl_input = to_input(gph_atl_weighted)

In [29]:
#Verify the structure of the input array is as expected
input_arr = wnp_input[300,2]
original_arr = gph_wnp_weighted.isel(case = 300, time = 0, latitude = 0, longitude = 2).values
print(input_arr, '\n', original_arr, sep = '')

3555.8125
3555.8125


In [30]:
#Initialize and xarray Dataset to hold the preprocessed data
ds_wnp_input = xr.Dataset(coords = {'case': np.array(wnp_metadata['wnp_id'])})
ds_atl_input = xr.Dataset(coords = {'case': np.array(atl_metadata['atl_id'])})

In [31]:
#Place the preprocessed data into the xarray Dataset
ds_wnp_input['gph']=(['case', 'values'], wnp_input)
ds_atl_input['gph']=(['case', 'values'], atl_input)

In [32]:
#Add some attributes
history = 'Values are weighted by the sqrt of the cos of the latitude'
ds_wnp_input.attrs['history'] = history
ds_atl_input.attrs['history'] = history

In [36]:
#Verify the Dataset to be output looks as expected
ds_atl_input

<xarray.Dataset>
Dimensions:  (case: 601, values: 58201)
Coordinates:
  * case     (case) object 'atl_001' 'atl_002' 'atl_003' ... 'atl_600' 'atl_601'
Dimensions without coordinates: values
Data variables:
    gph      (case, values) float32 3661.766 3662.651 ... 5309.957 5314.8633
Attributes:
    history:  Values are weighted by the sqrt of the cos of the latitude

In [38]:
ds_wnp_input.to_netcdf('som/wnp_som_input.nc')
ds_atl_input.to_netcdf('som/atl_som_input.nc')